# TensorBoard 測試

## 刪除 log 目錄

In [1]:
# 刪除 log 目錄
import os
import shutil

dirpath = '.\logs'
if os.path.exists(dirpath) and os.path.isdir(dirpath):
    shutil.rmtree(dirpath)

## 模型

In [2]:
import tensorflow as tf

mnist = tf.keras.datasets.mnist

# 載入 MNIST 手寫阿拉伯數字資料
(x_train, y_train),(x_test, y_test) = mnist.load_data()

# 特徵縮放，使用常態化(Normalization)，公式 = (x - min) / (max - min)
x_train_norm, x_test_norm = x_train / 255.0, x_test / 255.0

# 建立模型
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
])


## 設定優化器(optimizer)、損失函數(loss)、效能衡量指標(metrics)

In [3]:
# 設定優化器(optimizer)、損失函數(loss)、效能衡量指標(metrics)的類別
loss_function = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

# Define 訓練及測試的效能衡量指標(Metrics)
train_loss = tf.keras.metrics.Mean('train_loss', dtype=tf.float32)
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('train_accuracy')
test_loss = tf.keras.metrics.Mean('test_loss', dtype=tf.float32)
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('test_accuracy')

## 定義模型訓練的函數

In [4]:
def train_step(model, optimizer, x_train, y_train):
    # 自動微分
    with tf.GradientTape() as tape:
        predictions = model(x_train, training=True)
        loss = loss_function(y_train, predictions)
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

    # 計算訓練的效能衡量指標
    train_loss(loss)
    train_accuracy(y_train, predictions)

def test_step(model, x_test, y_test):
    # 預測
    predictions = model(x_test)
    # 計算損失
    loss = loss_function(y_test, predictions)

    # 計算測試的效能衡量指標
    test_loss(loss)
    test_accuracy(y_test, predictions)

## 設定 log 目錄，開啟 log 檔案

In [5]:
import datetime

current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
# 指定訓練的 log 檔名
train_log_dir = '.\\logs\\gradient_tape\\' + current_time + '\\train'
# 指定測試的 log 檔名
test_log_dir = '.\\logs\\gradient_tape\\' + current_time + '\\test'

# 開啟 log 檔案
train_summary_writer = tf.summary.create_file_writer(train_log_dir)
test_summary_writer = tf.summary.create_file_writer(test_log_dir)

## 將訓練/測試資料轉成 Tensorflow Dataset

In [6]:
# 將訓練/測試資料轉成 Tensorflow Dataset
train_dataset = tf.data.Dataset.from_tensor_slices((x_train_norm, y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((x_test_norm, y_test))

# 每次從 60000 筆訓練資料隨機抽出 64 筆
# shuffle：洗牌，batch：每批 64 筆
train_dataset = train_dataset.shuffle(60000).batch(64)
# 每次從 10000 筆測試資料隨機抽出 64 筆
test_dataset = test_dataset.batch(64)

## 模型訓練

In [7]:
EPOCHS = 5

# 訓練 5 次
for epoch in range(EPOCHS):
    # 訓練
    for (x_train, y_train) in train_dataset:
        train_step(model, optimizer, x_train, y_train)
        
    # 寫入訓練 log
    with train_summary_writer.as_default():
        tf.summary.scalar('loss', train_loss.result(), step=epoch)
        tf.summary.scalar('accuracy', train_accuracy.result(), step=epoch)

    # 測試
    for (x_test, y_test) in test_dataset:
        test_step(model, x_test, y_test)
        
    # 寫入測試 log
    with test_summary_writer.as_default():
        tf.summary.scalar('loss', test_loss.result(), step=epoch)
        tf.summary.scalar('accuracy', test_accuracy.result(), step=epoch)
    
    # 顯示結果
    template = 'Epoch {}, Loss: {}, Accuracy: {}%, Test Loss: {}, Test Accuracy: {}%'
    print(template.format(epoch+1,
         train_loss.result(), 
         train_accuracy.result()*100,
         test_loss.result(), 
         test_accuracy.result()*100))

    # 重置效能衡量指標
    train_loss.reset_states()
    test_loss.reset_states()
    train_accuracy.reset_states()
    test_accuracy.reset_states()


Epoch 1, Loss: 0.2851989269256592, Accuracy: 91.836669921875%, Test Loss: 0.13092444837093353, Test Accuracy: 96.01000213623047%
Epoch 2, Loss: 0.1275085210800171, Accuracy: 96.33000183105469%, Test Loss: 0.09767524152994156, Test Accuracy: 97.0199966430664%
Epoch 3, Loss: 0.08957860618829727, Accuracy: 97.29666900634766%, Test Loss: 0.07886812835931778, Test Accuracy: 97.47999572753906%
Epoch 4, Loss: 0.07196512818336487, Accuracy: 97.80332946777344%, Test Loss: 0.06988076865673065, Test Accuracy: 97.72999572753906%
Epoch 5, Loss: 0.056746840476989746, Accuracy: 98.2750015258789%, Test Loss: 0.06665626913309097, Test Accuracy: 97.79000091552734%


In [8]:
# 載入 TensorBoard notebook extension，即可在 jupyter notebook 啟動 Tensorboard
%load_ext tensorboard

In [9]:
# 啟動 Tensorboard
%tensorboard --logdir logs/gradient_tape

Launching TensorBoard...

KeyboardInterrupt: 

## 使用瀏覽器輸入以下網址，即可觀看訓練資訊：
## http://localhost:6006/

In [10]:
!taskkill /IM "tensorboard.exe" /F
# 或者使用以下指令，pid 以工作管理員查詢
# !taskkill /F /PID pid

成功: 處理程序 "tensorboard.exe" (PID 3336) 已經終止了。


## 寫入圖片

In [9]:
# 任意找一張圖片
img = x_train_norm[0].reshape((-1, 28, 28, 1))
img.shape

(1, 28, 28, 1)

In [10]:
import datetime

# 指定 log 檔名
logdir = ".\\logs\\train_data\\" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
# Creates a file writer for the log directory.
file_writer = tf.summary.create_file_writer(logdir)

# Using the file writer, log the reshaped image.
with file_writer.as_default():
    # 將圖片寫入 log 檔
    tf.summary.image("Training data", img, step=0)

In [28]:
%tensorboard --logdir logs/train_data

ERROR: Timed out waiting for TensorBoard to start. It may still be running as pid 17852.